<a href="https://colab.research.google.com/github/Shuraimi/DataScience-Handbook-Notes/blob/main/2.%20Data_manipulation_with_Pandas/8.Combining_Datasets_Merge_and_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combining Datasets : Merge and Join

In [ ]:
import numpy as np
import pandas as pd

One of the most useful operation in Pandas is the in-memory merge and join operations.

# Relational Algebra

The behaviour implemented in Pandas in `pd.merge()` is a subset of Relational Algebra which is a set of rules for manipulation of Relational data and forms the conceptual foundation of most operations in databases.

It proposes several primitive options that become building block of complex operations.

### Categories of Joins

The `pd.merge()` implements a number of joins
1. *one-to-one*
2. *many-to-one*
3. *many-to-many*

All these joins are accessed via an identical call to the `pd.merge()` interface and type of join depends on the form of input data.

#### One-to-one join

The most simplest join is the one-to-one join which is just like the column-wise Concatenation using `pd.concat()`.

An example:-

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
 'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
 'hire_date': [2004, 2008, 2012, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


We merge these two Dataframes into a single DataFrame using merge()

In [ ]:
df3=pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The `pd.merge()` automatically understands that that both the DataFrames have 'employee' column in common and joins the DataFrame on that column and results in a new DataFrame.

The order of entries in the columns doesn't matter and `pd.merge()` correctly accounts for this. The merge discards the index except in case of merges by index.

#### Many-to-one join

In this type of join, one of the two key columns contain duplicate entries. The resulting DataFrame will contain duplicates entries as appropriate.

**What this means???**

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
 'supervisor': ['Carly', 'Guido', 'Steve']})

In [ ]:

print(df3)
print(df4)
df5=pd.merge(df3,df4)
df5

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


The result has another column supervisor which contains one or more duplicates as required.

#### Many-to-many join

This join is conceptually confusing but defined. In this join, I'd key columns in both the left and right arrays contains duplicates then it is a many-to-many merge.

In [ ]:
df6 = pd.DataFrame({'group': ['Accounting', 'Accounting',
 'Engineering', 'Engineering', 'HR', 'HR'],
'skills': ['math', 'spreadsheets', 'coding', 'linux',
 'spreadsheets', 'organization']})

In [ ]:
print(df1)
print(df6)
print(pd.merge(df1,df6))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


We can get the skills of exam employee by performing the join.

These three types of joins provide a wide array of functionality. But the real world datasets are rarely clean.

In the next sections, we'll see few options that enable to tune the `pd.merge()' for various join operations.

## Specification of the Merge Key

The default behaviour of `pd.merge()` is that it looks for one or more matching column names in the given inputs and uses this as key. However, the column names often don't match and `pd.merge()` provides various methods to handle this.

### The on keyword

You an explicitly specify the key column using the *on* keyword which takes a column name or list of columns.

In [ ]:
print(df1)
print(df2)
print(pd.merge(df1,df2,on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


This option works only if both the DataFrames have the same column name.

### The left_on and right_on keywords

At times you might want to merge Dataframes with different column names using *left_on* and *right_on* keywords to specify column name.

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
 'salary': [70000, 80000, 120000, 90000]})

In [ ]:
print(df1)
print(df3)
print(pd.merge(df1,df3,left_on='employee',right_on='name'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


The result has redundant column which can be removed by using the `drop()` of Dataframes.

In [ ]:
print(pd.merge(df1,df3,left_on='employee',right_on='name').drop('name',axis=1))

  employee        group  salary
0      Bob   Accounting   70000
1     Jake  Engineering   80000
2     Lisa  Engineering  120000
3      Sue           HR   90000


### The left_index and right_index keyword

Sometimes rather than merging on columns, we can also merge on index like 👇

In [ ]:
df1a=df1.set_index('employee')
df2a=df2.set_index('employee')

print(df1a)
print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


You can use this index to merge Dataframes by specifying the left_index and/or right_index flags in `pd.merge()`.

In [ ]:
print(pd.merge(df1a,df2a,left_index=True,right_index=True))

In [ ]:
print(pd.merge(df1a,df2a,left_index=True,right_index=False))

MergeError: ignored

This error says that you must either specify the `left_index=True` or left_on and same for `right_index=True` or right_on

For convenience, Dataframes implement the `join()` that performs a merge  that defaults to joining.

In [ ]:
print(df1a.join(df2a))

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


If you want to mix indices and columns, you can combine *left_index* with *right_on* or *right_index* with *left_on* to get desired result.

In [ ]:
print(df1a)
print(df3)
print(pd.merge(df1a,df3,left_index=True,right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


This code merges the left index of df1a on the column name of df3.

This approach of Indexing can be done for multiple indices and/or with multiple columns. Refer to documentation for more information.

## Specifying Set arithmetic for joins

In all the preceding examples discussed, we haven't included the type of set arithmetic to perform join. This happens if a value appears in one key column and not in the other.

In [ ]:
df7= pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],'food': ['fish', 'beans', 'bread']},columns=['name', 'food'])
df8 = pd.DataFrame({'name': ['Mary', 'Joseph'],'drink': ['wine', 'beer']},columns=['name', 'drink'])
print(df7)
print(df8)

print(pd.merge(df7, df8))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


The result contains only one record since both the sets for merge have only one value in common. By default, the merge performs intersection of these sets and this is what is called the *inner* join.

This is explicitly specified by the *how* keyword

In [ ]:
pd.merge(df7,df8,how='inner')

,name,food,drink
0,Mary,bread,wine


Other options are *outer* , *left* and *right*. The *outer* keyword specifies the union of input columns and fill missing values with NA.

In [ ]:
pd.merge(df7,df8,how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


The *left join* and *right join* returns the join over left and right entries respectively.

In [ ]:
pd.merge(df7,df8,how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


The output rows corresponds to entries in the left input and similarly for right join.

In [ ]:
pd.merge(df7,df8,how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


## Overlapping Column name : Suffixes keyword

Finally you can end up having conflicting names of the 2 input Dataframes.

**Check this out again**

In [ ]:
df9= pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
 'rank': [1, 2, 3, 4]})
df10 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
 'rank': [3, 1, 4, 2]})

print(pd.merge(df9,df10,on='name'))

   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


The result contains conflicting columns that are suffixed with _x and _y to make the columns unique. We can also specify the custom suffix using the suffixes keyword.

In [ ]:
print(pd.merge(df9,df10,on='name',suffixes=['_L','_R']))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


## US States Data

In [ ]:
areas_df=pd.read_csv('/content/drive/MyDrive/Data Science /state-areas.csv')
pop_df=pd.read_csv('/content/drive/MyDrive/Data Science /state-population.csv')
abbrev_df=pd.read_csv('/content/drive/MyDrive/Data Science /state-abbrevs.csv')

In [ ]:
areas_df.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [ ]:
pop_df.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [ ]:
abbrev_df.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


We want to rank the US States, but first we merge the datasets pop_df and abbrev_df

In [ ]:
merged=pd.merge(pop_df,abbrev_df,how='outer',left_on='state/region',right_on='abbreviation')

In [ ]:
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


What we did above is performed outer join on the datasets  ut since the Datsets have different column names, we used left_on and right_on keywords.

Drop the abbreviation column since it's repeated.

In [ ]:
merged.drop(['abbreviation'],axis=1,inplace=True)
merged

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


Now we check if there are any mismatches i.e null values

In [ ]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the population and states data is missing let's check this out

In [ ]:
merged[merged.population.isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


The null values are from Puerto Rico before the year 2000. This is due to data missing in original source.

Now the states column.

In [ ]:
merged[merged.state.isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

The PR and USA do not have their corresponding abbreviations in the abbrevs DataFrame. Therefore we add them

In [ ]:
merged.loc[merged['state/region']=='PR','state']='Puerto Rico'

In [ ]:
merged.loc[merged['state/region']=='USA','state']='United States'

In [ ]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Now merge the states dataset.

In [ ]:
final=pd.merge(merged,areas_df,on='state',how='outer')

In [ ]:
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [ ]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool